<a href="https://www.kaggle.com/code/hasib111/optimize-pdf?scriptVersionId=295535453" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!apt-get -qq update && apt-get -qq install -y ghostscript

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
from pathlib import Path
import subprocess, shlex

pdf_in = Path("/kaggle/input/document/B25-6198_959-Sebastopol-Rd_Kitchen_PLN.pdf")
san = Path("/kaggle/working/sanitized.pdf")
opt = Path("/kaggle/working/optimized.pdf")


In [3]:
def run(cmd):
    print("Running:", " ".join(shlex.quote(c) for c in cmd))
    p = subprocess.run(cmd, capture_output=True, text=True)
    if p.returncode != 0:
        print(p.stdout); print(p.stderr)
        raise RuntimeError("Command failed")
    return p


In [4]:
# 1) Sanitize (parser-friendly)
run([
    "gs","-dNOPAUSE","-dBATCH","-dSAFER",
    "-sDEVICE=pdfwrite","-dCompatibilityLevel=1.4",
    "-dPDFSETTINGS=/prepress",
    "-o", str(san),
    str(pdf_in)
])

# 2) Optimize (shrink)
run([
    "gs","-dNOPAUSE","-dBATCH","-dSAFER",
    "-sDEVICE=pdfwrite","-dCompatibilityLevel=1.4",
    "-dDetectDuplicateImages=true",
    "-dCompressFonts=true","-dSubsetFonts=true",
    "-dPDFSETTINGS=/ebook",   # try /printer or /screen too
    "-o", str(opt),
    str(san)
])

print("\nSizes:")
print("original :", pdf_in.stat().st_size, "bytes")
print("sanitized:", san.stat().st_size, "bytes")
print("optimized:", opt.stat().st_size, "bytes")
print("\nOutput:", opt)

Running: gs -dNOPAUSE -dBATCH -dSAFER -sDEVICE=pdfwrite -dCompatibilityLevel=1.4 -dPDFSETTINGS=/prepress -o /kaggle/working/sanitized.pdf /kaggle/input/document/B25-6198_959-Sebastopol-Rd_Kitchen_PLN.pdf
Running: gs -dNOPAUSE -dBATCH -dSAFER -sDEVICE=pdfwrite -dCompatibilityLevel=1.4 -dDetectDuplicateImages=true -dCompressFonts=true -dSubsetFonts=true -dPDFSETTINGS=/ebook -o /kaggle/working/optimized.pdf /kaggle/working/sanitized.pdf

Sizes:
original : 19556446 bytes
sanitized: 36653480 bytes
optimized: 10606544 bytes

Output: /kaggle/working/optimized.pdf
